# Préparation pour le Machine Learning

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

df_movies = pd.read_csv("../BD_A_IGNORE/movies.csv", sep=',', index_col=0)
df_movies = df_movies.drop(columns=["level_0", "index"])
df_movies = df_movies[df_movies['titleType'] == "movie"]
df_movies.head(3)

,tconst,titleType,originalTitle,isAdult,startYear,runtimeMinutes,genres,directors,writers,averageRating,...,spoken_languages,vote_average,vote_count,actors_name,actors_rank,directors_name,writers_name,producers_name,cinematographers_name,editors_name
91,tt0000574,movie,The Story of the Kelly Gang,False,1906,70.0,"Action,Adventure,Biography",nm0846879,nm0846879,6.0,...,['xx'],5.375,28,"['Elizabeth Tait', 'Bella Cola', 'Charles Tait']","[1, 6, 11]",['Charles Tait'],['Charles Tait'],"['W.A. Gibson', 'Millard Johnson', 'John Tait'...","['Millard Johnson', 'Orrie Perry', 'Reg Perry']",Unknown
97,tt0000591,movie,L'enfant prodigue,False,1907,90.0,Drama,nm0141150,nm0141150,5.7,...,['xx'],0.000,0,"['Christiane Mandelys', 'Gilberte Sergy', 'Mic...","[3, 4, 5]",['Michel Carré'],['Michel Carré'],Unknown,Unknown,Unknown
241,tt0001184,movie,Don Juan de Serrallonga,False,1910,58.0,"Adventure,Drama","nm0063413,nm0550220",nm0049370,3.8,...,['xx'],1.000,1,"['Dolores Puchol', 'Ricardo de Baños', 'Albert...","[1, 3, 4]","['Ricardo de Baños', 'Alberto Marro']",['Víctor Balaguer'],Unknown,['Ramón de Baños'],Unknown


In [3]:
df_movies['titleType'].value_counts()

titleType
movie    191270
Name: count, dtype: int64

In [4]:
# informations générales de la base
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 191270 entries, 91 to 235135
Data columns (total 30 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   tconst                 191270 non-null  object 
 1   titleType              191270 non-null  object 
 2   originalTitle          191270 non-null  object 
 3   isAdult                191270 non-null  bool   
 4   startYear              191270 non-null  int64  
 5   runtimeMinutes         191270 non-null  float64
 6   genres                 191270 non-null  object 
 7   directors              191270 non-null  object 
 8   writers                191270 non-null  object 
 9   averageRating          191270 non-null  float64
 10  numVotes               191270 non-null  float64
 11  periode                191270 non-null  int64  
 12  budget                 191270 non-null  int64  
 13  id                     191270 non-null  int64  
 14  original_language      191270 non-null  

In [5]:
# check valeurs manquantes
df_movies.isna().sum()

tconst                   0
titleType                0
originalTitle            0
isAdult                  0
startYear                0
runtimeMinutes           0
genres                   0
directors                0
writers                  0
averageRating            0
numVotes                 0
periode                  0
budget                   0
id                       0
original_language        0
overview                 0
popularity               0
poster_path              0
production_countries     0
revenue                  0
spoken_languages         0
vote_average             0
vote_count               0
actors_name              0
actors_rank              0
directors_name           0
writers_name             0
producers_name           0
cinematographers_name    0
editors_name             0
dtype: int64

In [6]:
# descriptif statistiques
df_movies.describe()

,startYear,runtimeMinutes,averageRating,numVotes,periode,budget,id,popularity,revenue,vote_average,vote_count
count,191270.000000,191270.000000,191270.000000,1.912700e+05,191270.000000,1.912700e+05,1.912700e+05,191270.000000,1.912700e+05,191270.000000,191270.000000
mean,1994.195399,94.972400,6.022176,5.985674e+03,1989.746798,1.379140e+06,3.549511e+05,3.511111,3.615819e+06,4.501696,100.637800
std,26.603984,21.916737,1.218017,4.692797e+04,26.836747,1.034081e+07,2.978892e+05,26.745629,3.832356e+07,2.743128,744.444926
min,1903.000000,1.000000,1.000000,5.000000e+00,1910.000000,0.000000e+00,2.000000e+00,0.600000,0.000000e+00,0.000000,0.000000
25%,1977.000000,83.000000,5.300000,4.600000e+01,1970.000000,0.000000e+00,9.832875e+04,0.731000,0.000000e+00,2.400000,1.000000
50%,2005.000000,92.000000,6.200000,1.770000e+02,2000.000000,0.000000e+00,2.805165e+05,1.365000,0.000000e+00,5.400000,4.000000
75%,2016.000000,104.000000,6.800000,7.930000e+02,2010.000000,0.000000e+00,5.364738e+05,2.655000,0.000000e+00,6.500000,14.000000
max,2025.000000,240.000000,10.000000,2.959184e+06,2020.000000,5.793304e+08,1.122413e+06,5089.969000,2.923706e+09,10.000000,33630.000000


## Transformation de la colonne "genres" : extraire les genres uniques et les transformer en colonnes booléenes

#### Fonction transformant la chaine de films en liste et création de la colonne genres_liste

In [7]:
def split_chaine_en_liste(x):
    if isinstance(x, str): # si x est une chaine de caractère, 
        return x.split(',') # applique le split pour transformer la chaine en liste de plusieurs chaines de caractères
    else:
        return x # sinon retourne x tel quel


In [8]:
df_movies['genres_liste'] = df_movies['genres'].apply(split_chaine_en_liste)
df_movies['genres_liste']

91        [Action, Adventure, Biography]
97                               [Drama]
241                   [Adventure, Drama]
323                            [Unknown]
346                              [Drama]
                       ...              
235131                          [Family]
235132                     [Documentary]
235133     [Action, Adventure, Thriller]
235134                        [Thriller]
235135                  [Drama, History]
Name: genres_liste, Length: 191270, dtype: object

#### Extraction des genres uniques dans toutes les listes

In [9]:
# extraire les genres uniques 
tous_les_genres = set()
for genres in df_movies['genres_liste']:
  tous_les_genres.update(genres)

tous_les_genres

{'Action',
 'Adult',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Film-Noir',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Sport',
 'Talk-Show',
 'Thriller',
 'Unknown',
 'War',
 'Western'}

#### Création d'autant de colonnes binaires que de genres uniques

In [10]:
import warnings
warnings.filterwarnings("ignore") 

# créer des colonnes binaires pour chaque genre unique
for genre in tous_les_genres:
    def genre_present(x):
        return int(genre in x)
    
    df_movies[f'genre_{genre}'] = df_movies['genres_liste'].apply(genre_present)

df_movies.columns

Index(['tconst', 'titleType', 'originalTitle', 'isAdult', 'startYear',
       'runtimeMinutes', 'genres', 'directors', 'writers', 'averageRating',
       'numVotes', 'periode', 'budget', 'id', 'original_language', 'overview',
       'popularity', 'poster_path', 'production_countries', 'revenue',
       'spoken_languages', 'vote_average', 'vote_count', 'actors_name',
       'actors_rank', 'directors_name', 'writers_name', 'producers_name',
       'cinematographers_name', 'editors_name', 'genres_liste', 'genre_Comedy',
       'genre_Adventure', 'genre_Western', 'genre_Unknown', 'genre_Mystery',
       'genre_Biography', 'genre_Sci-Fi', 'genre_Animation',
       'genre_Reality-TV', 'genre_Romance', 'genre_Music', 'genre_Family',
       'genre_Adult', 'genre_Drama', 'genre_Thriller', 'genre_Game-Show',
       'genre_Fantasy', 'genre_Documentary', 'genre_Film-Noir',
       'genre_History', 'genre_Musical', 'genre_Talk-Show', 'genre_Sport',
       'genre_Action', 'genre_News', 'genre_War', '

In [11]:
df_movies.sample(5)

,tconst,titleType,originalTitle,isAdult,startYear,runtimeMinutes,genres,directors,writers,averageRating,...,genre_Film-Noir,genre_History,genre_Musical,genre_Talk-Show,genre_Sport,genre_Action,genre_News,genre_War,genre_Crime,genre_Horror
37050,tt0072430,movie,Yojôhan fusuma no urabari,False,1973,72.0,"Comedy,Drama",nm0436770,"nm0436770,nm0619079",5.5,...,0,0,0,0,0,0,0,0,0,0
31358,tt0062893,movie,Dillinger è morto,False,1969,95.0,"Crime,Drama",nm0274659,"nm0274659,nm0063376",6.9,...,0,0,0,0,0,0,0,0,1,0
71900,tt0185434,movie,Locos por la música,False,1980,90.0,"Comedy,Musical",nm0205955,nm0763126,4.6,...,0,0,1,0,0,0,0,0,0,0
81794,tt0246719,movie,Julie Walking Home,False,2002,118.0,"Drama,Romance",nm0002140,"nm0002140,nm0340180,nm0765403",6.3,...,0,0,0,0,0,0,0,0,0,0
213369,tt6116856,movie,The Night Comes for Us,False,2018,121.0,"Action,Crime,Thriller",nm3464186,nm3464186,6.9,...,0,0,0,0,0,1,0,0,1,0


In [ ]:
import joblib
joblib.dump(df_movies, "../BD_A_IGNORE/df_movies.pkl") # enregistrer la base sans utiliser de csv

['df_movies.pkl']

# Base données pour ML

#### Encodage des colonnes pour entrainement

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)  # limite à 5000 mots pour éviter trop de dimensions
tfidf_matrix = tfidf_vectorizer.fit_transform(df_movies['overview'])
tfidf_matrix.shape


(191270, 5000)

#### Choix des variables et initialisation 

In [ ]:
from scipy.sparse import hstack

features = ['runtimeMinutes', 'averageRating', 'numVotes', 'popularity', 'budget'] + \
           [col for col in df_movies.columns if col.startswith('genre_')]

# encodage des valeurs numériques
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_movies[features])

X_final = hstack([X_scaled, tfidf_matrix]) # fusionner les caractéristiques numériques et la matrice TF-IDF
X_final.shape

(191270, 5033)

# Créer et entraîner le modèle KNN

In [16]:
knn = NearestNeighbors(n_neighbors=10, metric='cosine')  # On utilise la similarité cosinus pour le texte
knn.fit(X_final)

NearestNeighbors(metric='cosine', n_neighbors=10)

# Fonction de recommandation des films

In [ ]:
def recommander_films(film_titre, df_movies, features, scaler, tfidf_vectorizer, knn):
    """
    Trouve des films similaires à celui donné en entrée en utilisant KNN.
    
    - film_titre : titre du film de référence
    - df_movies : DataFrame contenant les informations des films
    - df_movies_new : DataFrame utilisé pour entraîner le modèle KNN
    - features : colonnes utilisées pour KNN
    - scaler : StandardScaler entraîné
    - tfidf_vectorizer : modèle TF-IDF entraîné
    - knn : modèle KNN entraîné
    """
    
    # vérifier si le film est bien dans la base
    if film_titre not in df_movies['originalTitle'].values:
        return "Film non trouvé dans la base."

    # Trouver l’index du film dans df_movies
    film_index_movies = df_movies[df_movies['originalTitle'] == film_titre].index[0]

    # Extraire ses caractéristiques et standardiser
    film_features = df_movies.loc[film_index_movies, features].values.reshape(1, -1)
    film_features_scaled = scaler.transform(film_features)

    # Transformer `overview` en vecteur TF-IDF
    film_overview = df_movies.loc[film_index_movies, 'overview']
    film_overview_tfidf = tfidf_vectorizer.transform([film_overview])

    # Fusionner les caractéristiques numériques et le TF-IDF
    film_vector = hstack([film_features_scaled, film_overview_tfidf])

    # Trouver les films les plus proches
    distances, indices = knn.kneighbors(film_vector)

    # Récupérer les films recommandés
    recommandations = df_movies.iloc[indices[0][1:]]  # Exclure le film lui-même

    # Retourner les films avec plus d'infos (titre, période, genres, rating, popularité)
    return recommandations[['originalTitle', 'periode', 'averageRating', 'popularity'] + [col for col in df_movies.columns if col.startswith('genre_')]]


In [18]:
verif = df_movies[df_movies['originalTitle'] == 'The Last Tycoon']
verif

,tconst,titleType,originalTitle,isAdult,startYear,runtimeMinutes,genres,directors,writers,averageRating,...,genre_Film-Noir,genre_History,genre_Musical,genre_Talk-Show,genre_Sport,genre_Action,genre_News,genre_War,genre_Crime,genre_Horror
38220,tt0074777,movie,The Last Tycoon,False,1976,123.0,"Drama,Romance",nm0001415,"nm0280234,nm0056217",6.2,...,0,0,0,0,0,0,0,0,0,0


In [19]:
recommander_films("The Last Tycoon", df_movies, df_movies_new, features, scaler, tfidf_vectorizer, knn)

,originalTitle,periode,averageRating,popularity,genre_Comedy,genre_Adventure,genre_Western,genre_Unknown,genre_Mystery,genre_Biography,...,genre_Film-Noir,genre_History,genre_Musical,genre_Talk-Show,genre_Sport,genre_Action,genre_News,genre_War,genre_Crime,genre_Horror
42904,Identificazione di una donna,1980,6.7,3.769,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
176832,The Mistress,2010,6.4,2.126,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
57629,Moll Flanders,1990,6.4,4.832,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68851,L'ennui,1990,6.0,6.937,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24259,Marjorie Morningstar,1950,6.2,1.954,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91869,AKA,2000,6.3,8.218,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
217696,Sultana Bibiana,2010,6.1,0.600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
124925,Your Love Song,2020,6.2,2.828,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59323,A Price Above Rubies,1990,6.5,4.437,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
